In [1]:
import schedule
import time
from email_processor.email.reader import EmailReader
from email_processor.llm.processor import LLMProcessor
from email_processor.database.repository import DatabaseRepository


class EmailProcessor:
    """Class to handle email processing using LLM and database operations."""

    def __init__(self):
        """Initialize the EmailProcessor with EmailReader, LLMProcessor, and DatabaseRepository."""
        self.reader = EmailReader()
        self.llm = LLMProcessor()
        self.db = DatabaseRepository()
        if not self.reader.connect():
            print("Failed to connect to the email server.")
            return

    def process_one_email(self):
        """
        Fetch one email, process it using the LLM, and store it in the database.
        """

        print("Connected to email server.")
        print("Fetching emails...")
        emails = self.reader.fetch_emails(count=1)  # Fetch one email
        if not emails:
            print("No emails fetched.")
            return

        print("Processing email...")
        email_data = emails[0]  # Process the first email
        processed_data = self.llm.process(email_data)  # Process email using LLM
        print("Inserting processed email into the database...")
        self.db.insert_processed_email(processed_data)  # Insert into database
        print("Email processed and stored successfully.")

    def run_scheduled(self, interval_minutes=60):
        """
        Run the email processing task on a scheduled basis.
        :param interval_minutes: Interval in minutes for scheduling the task.
        """
        schedule.every(interval_minutes).minutes.do(self.process_one_email)
        print(f"Scheduled email processing to run every {interval_minutes} minutes.")

        while True:
            schedule.run_pending()
            time.sleep(1)

2025-01-19 15:17:46,057 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.


In [2]:
processor = EmailProcessor()
processor.process_one_email()


Connected to email server.
Fetching emails...
Processing email...
Raw API Response: ```json
{
  "junk": false,
  "product_issue": false,
  "personal_issue": false,
  "delivery_issue": false,
  "payment_issue": false,
  "priority": 2
}
```
Inserting processed email into the database...
Email processed and stored successfully.


In [3]:
proc2 = EmailProcessor()
proc2.process_one_email()

Connected to email server.
Fetching emails...


/opt/homebrew/Caskroom/miniconda/base/envs/jane_street/lib/python3.12/email/_header_value_parser.py:140: RuntimeWarning: coroutine 'async_generator_to_list' was never awaited
  return sum((x.all_defects for x in self), self.defects)


Processing email...
Raw API Response: ```json
{
  "junk": true,
  "product_issue": false,
  "personal_issue": true,
  "delivery_issue": false,
  "payment_issue": false,
  "priority": 3
}
```
Inserting processed email into the database...
Existing email found.
Email processed and stored successfully.
